<a href="https://colab.research.google.com/github/cellphone0966644265/storage-samples/blob/main/download_to_onedrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Install Rclone and Upload File Rclone.config<img src='https://biplobsd.github.io/RLabClone/img/title_rclonelab.svg' height="45" alt="RcloneLab"/>

rclone.config file

1 download rclone https://rclone.org/downloads/

2 create rclone config file https://www.youtube.com/watch?v=Sp9lG_BYlSg

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!wget https://downloads.rclone.org/v1.52.1/rclone-v1.52.1-linux-amd64.deb
!apt install ./rclone-v1.52.1-linux-amd64.deb
import libtorrent as lt
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
# @title Install Rclone and Upload File Rclone.config {vertical-output: true}
Setup_Time_Zone = False
import os; from google.colab import files; from IPython.display import HTML, clear_output
def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#f9bd0d;\">Vui lòng upload file (rclone.conf) từ máy tính của bạn.</h2><br>"))
    UploadConfig = files.upload().keys()
    clear_output(wait=True)
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Tải lên tệp đã bị hủy trong khi tải tệp lên.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn) == True:
          os.environ["rclone_conf"] = fn
          !mv -f "$rclone_conf" /root/.rclone.conf
          !chmod 666 /root/.rclone.conf
          if Setup_Time_Zone == True:
            !sudo dpkg-reconfigure tzdata
            clear_output(wait=True)
          if os.path.isfile("/usr/bin/rclone") == True:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#f50e02;\">Cấu hình đã được thay đổi.</h2><br></center>"))
          else:
            !rm -rf /content/sample_data/
            !curl -s https://rclone.org/install.sh | sudo bash
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#f9bd0d;\">Quá trình cài đặt đã hoàn tất thành công.</h2><br></center>"))
        else:
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#f50e02;\">Tải lên tệp không thành công trong quá trình tải tệp lên.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        os.environ["rclone_conf"] = fn
        !rm -f "$rclone_conf"
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#f50e02;\">Vui lòng chỉ tải lên một tệp mỗi lần.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#f50e02;\">Đã xảy ra lỗi trong quá trình tải tệp lên.</h2><br></center>"))

upload_conf()
!mkdir onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: ./onedrive &

# 2. <font color='orange'>Check Mounted

In [ ]:
# @title Check mount { vertical-output: true }
!ls /content/onedrive/

# 3. <font color='orange'>Download

In [ ]:
# @title add url to download { vertical-output: true }
file_name = input ("File name.extention")
location = input("Locattion:")
url = input("URL")
!wget -O /content/onedrive/"$location"/"$file_name" "$url"


# 4. <font color='orange'>Download Torrent/Magnet Link

In [ ]:
# @title Add from torrent file { vertical-output: true }
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/onedrive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

In [ ]:
# @title Add from magnet link { vertical-output: true }
params = {"save_path": "/content/onedrive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


In [ ]:
# @title Start download { vertical-output: true }
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)
